In [1]:
import neo4j
import pandas as pd
from utils import simplify_to_centroid_if_small
from utils_neo4j import init, station_insert_query
from geofox_client import get_geofox_client
from utils_geofox import get_stations, stationdf2rows, get_departures
import pandas as pd

In [2]:
NEO4J_URI = "bolt://localhost:7689"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "busticket123"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
init(driver)

In [3]:
client = get_geofox_client()

In [4]:
# Fetch the POIs
stations = get_stations(client, loadFromDisk=True)['stations']

In [5]:
stations_hamburg = [s for s in stations if s['city'] == 'Hamburg']

In [6]:
len(stations_hamburg), len(stations)

(2307, 8828)

In [7]:
stations_hamburg_req = [{"id" : s['id'], "type": "STATION", "name": s["name"]} for s in stations_hamburg]

In [8]:
# iterate over the stations in step size 5
for i in range(0, len(stations_hamburg_req), 5):
    sub_req = stations_hamburg_req[i:i+5]
    # set file name to join of the ids
    file_name = f"{i}_{i+5}.json"
    print(f"Fetching departures for {file_name} ...")
    res = get_departures(client, sub_req, filename=file_name)
    print(f"Fetched {len(res['departures'])} departures")
    break


Fetching departures for 0_5.json ...
Fetched 4956 departures


In [13]:
test_stationids

[{'id': 'Master:51334', 'type': 'STATION', 'name': 'Petersberg'},
 {'id': 'Master:51401', 'type': 'STATION', 'name': 'Marxener Straße'},
 {'id': 'Master:51402', 'type': 'STATION', 'name': 'Büntestraße'},
 {'id': 'Master:51403', 'type': 'STATION', 'name': 'Hauptstraße'},
 {'id': 'Master:51404', 'type': 'STATION', 'name': 'Vor dem Haßel'}]

In [6]:
test2 = [{ 
  "name": "Rosenhof", 
  "city": "Ahrensburg", 
  "combinedName": "Ahrensburg, Rosenhof", 
  "id": "Master:35009", 
  "type": "STATION", 
 }]

In [10]:
departures = get_departures(client, test_stationids)

In [14]:
len(departures['departures'])

178

In [7]:
impcols = ['id', 'name', 'city', 'vehicleTypes', 'coordinate', 'aliasses']

In [8]:
df_stations_impcols = df_stations[impcols]

In [9]:
df_stations_impcols.describe()

,id,name,city,vehicleTypes,coordinate,aliasses
count,8828,8828,8828,8828,8828,2976
unique,8828,7083,1098,32,8828,2976
top,Master:90650901,Schule,Hamburg,[REGIONALBUS],"{'x': 9.536332, 'y': 53.842018}",[Kleve (Itzehoe) Abzw. Rahde]
freq,1,91,2307,5453,1,1


In [10]:
rows = stationdf2rows(df_stations_impcols)

In [11]:
rows

[{'geofoxid': 'Master:10000',
  'name': 0,
  'city': 'Hamburg',
  'vehicleTypes': ['REGIONALBUS', 'NACHTBUS'],
  'aliasses': [],
  'longitude': 10.042491,
  'latitude': 53.534193},
 {'geofoxid': 'Master:10001',
  'name': 1,
  'city': 'Hamburg',
  'vehicleTypes': ['REGIONALBUS', 'XPRESSBUS'],
  'aliasses': ['Hauptbahnhof/Steintorwall'],
  'longitude': 10.005926,
  'latitude': 53.55113},
 {'geofoxid': 'Master:10002',
  'name': 2,
  'city': 'Hamburg',
  'vehicleTypes': ['REGIONALBUS', 'NACHTBUS', 'XPRESSBUS'],
  'aliasses': ['Hamburg ZOB', 'Hbf ZOB'],
  'longitude': 10.010335,
  'latitude': 53.551901},
 {'geofoxid': 'Master:10003',
  'name': 3,
  'city': 'Hamburg',
  'vehicleTypes': ['REGIONALBUS', 'NACHTBUS', 'XPRESSBUS'],
  'aliasses': [],
  'longitude': 10.023077,
  'latitude': 53.553464},
 {'geofoxid': 'Master:10004',
  'name': 4,
  'city': 'Hamburg',
  'vehicleTypes': ['REGIONALBUS', 'XPRESSBUS'],
  'aliasses': ['U-Bahnhof S-Bahnhof Berliner Tor'],
  'longitude': 10.025576,
  'latitu

In [12]:
# Execute the query
with driver.session() as session:
    result = session.run(station_insert_query, rows=rows)
    print(f"Imported {result.single()['total']} stations")

driver.close()

Imported 8828 stations
